In [ ]:
import sys
from pathlib import Path

sys.path.append("..")
from ribasim_lumping import RibasimLumpingNetwork

import pandas as pd
import numpy as np
import geopandas as gpd
from pydantic import BaseModel
import xarray as xr
import dfm_tools as dfmt
import xugrid as xu
import matplotlib.pyplot as plt
import random
from shapely.geometry import LineString

from numba.core.errors import NumbaDeprecationWarning, NumbaPendingDeprecationWarning
import warnings

warnings.simplefilter("ignore", category=NumbaDeprecationWarning)

import networkx as nx
import ribasim

In [ ]:
%load_ext autoreload
%autoreload 2

Define base directory and results directory

In [ ]:
base_dir = Path("..\\..\\ribasim_lumping_data\\")
results_dir = Path(base_dir, "results")

Define network name and load areas (discharge units: afwaterende eenheden)

In [ ]:
network_name = "zutphen_tki_netwerk"

areas_file_path = Path(base_dir, "afw_eenheden\\wrij_afwateringseenheden_clip_Zutphen.shp")
areas_gdf = gpd.read_file(areas_file_path)

Create networkanalysis

In [ ]:
network = RibasimLumpingNetwork(name=network_name, areas_gdf=areas_gdf)

Select simulation sets and extract all data points

In [ ]:
network.add_data_from_simulations_set(
    set_name="winter",
    simulations_dir=Path(base_dir, "d-hydro\\"),
    simulations_names=["tki_zuthpen_berkel_basis.dsproj"],
    simulations_ts=pd.date_range("2000-01-02 23:00", periods=9, freq="2D"),
)
network.add_data_from_simulations_set(
    set_name="zomer",
    simulations_dir=Path(base_dir, "d-hydro\\"),
    simulations_names=["tki_zuthpen_berkel_basis.dsproj"],
    simulations_ts=pd.date_range("2000-01-02 23:00", periods=9, freq="2D"),
)

Read network data and extract all objects (weirs/pumps/laterals/confluences/bifurcations)

In [ ]:
network.get_network_data()

In [ ]:
# network.nodes_gdf[network.nodes_gdf.mesh1d_node_id.isin(['1334'])]

Define node_ids on which to split the network into Ribasim basins:
- define types to include
- define additional split nodes by id
- define which of the node_ids should be excluded
- combine types of split_node_ids
- add split_nodes to network

In [ ]:
split_node_ids = network.get_node_ids_from_type(
    bifurcations=False,
    confluences=False,
    weirs=True,
    pumps=True,
    laterals=False,
)
split_node_ids_to_include = [1452, 2378, 419, 96]
split_node_ids_to_exclude = [314]

split_node_ids = [node_id for node_id in split_node_ids + split_node_ids_to_include 
                  if node_id not in split_node_ids_to_exclude]

network.add_split_nodes_based_on_node_ids(split_node_ids=split_node_ids);

Create basins (gdf) based on nodes, edges, split_node_ids and areas

In [ ]:
network.create_basins_based_on_split_nodes();

Find and create ribasim_edges_gdf between basins

In [ ]:
network.basins_gdf.head(3)

In [ ]:
network.split_nodes.head(3)

In [ ]:
network.nodes_gdf.head(3)

In [ ]:
network.edges_gdf.head(3)

In [ ]:

ribasim_edges_gdf = network.split_nodes[['mesh1d_nNodes','geometry']]
ribasim_edges_gdf = ribasim_edges_gdf.rename(columns={"geometry":"geometry_splitnode"})

# merge splitnodes with edges
ribasim_edges_ds = ribasim_edges_gdf.merge(network.edges_gdf[['basin', 'start_node_no', 'end_node_no','mesh1d_nEdges']], left_on='mesh1d_nNodes', right_on='start_node_no')
ribasim_edges_us = ribasim_edges_gdf.merge(network.edges_gdf[['basin', 'start_node_no','end_node_no','mesh1d_nEdges']], left_on='mesh1d_nNodes', right_on='end_node_no')

# DS
# merge splitnodes with basins
ribasim_edges_ds = ribasim_edges_ds.merge(network.basins_gdf[['basin', 'geometry']], left_on='basin', right_on='basin').rename(columns={"geometry":"geometry_basin"})
ribasim_edges_ds['direction'] = 'in'
basin_connections_ds = ribasim_edges_ds.copy()

# # draw connection
# from shapely.geometry import LineString
# ribasim_edges_ds['geometry'] = ribasim_edges_ds.apply(lambda row: LineString([row['geometry_splitnode'], row['geometry_basin']]), axis=1)
# ribasim_edges_ds = gpd.GeoDataFrame(ribasim_edges_ds, geometry='geometry', crs=28992)


# US
# merge splitnodes with basins
ribasim_edges_us = ribasim_edges_us.merge(network.basins_gdf[['basin', 'geometry']], left_on='basin', right_on='basin').rename(columns={"geometry":"geometry_basin"})
ribasim_edges_us['direction'] = 'out'
basin_connections_us = ribasim_edges_us.copy()

# # draw connection
# ribasim_edges_us['geometry'] = ribasim_edges_us.apply(lambda row: LineString([row['geometry_basin'],row['geometry_splitnode']]), axis=1)
# ribasim_edges_us = gpd.GeoDataFrame(ribasim_edges_us, geometry='geometry', crs = 28992)

# # concat us and ds
# ribasim_edges_gdf = pd.concat([ribasim_edges_ds, ribasim_edges_us])
# # basin_connections = ribasim_edges_gdf.copy()

# ribasim_edges_gdf = ribasim_edges_gdf.drop(columns=['geometry_splitnode','geometry_basin'])

# merge basin connections with nodes
basin_connections_us = basin_connections_us.merge(network.nodes_gdf[['mesh1d_nNodes', 'geometry']], left_on='start_node_no', right_on='mesh1d_nNodes', suffixes=('','_r')).rename(columns={"geometry":"geometry_edge_start_node"})
basin_connections_ds = basin_connections_ds.merge(network.nodes_gdf[['mesh1d_nNodes', 'geometry']], left_on='end_node_no', right_on='mesh1d_nNodes', suffixes=('','_r')).rename(columns={"geometry":"geometry_edge_end_node"})
# basin_connections_us['x'] = basin_connections_us.geometry_edge_start_node.x

basin_connections_us.head(3)

In [ ]:
# basin_connections_us['coords'] = basin_connections_us.geometry_basin.apply(lambda p: list(p.coords)[0])
# basin_connections_us['coords'] = (basin_connections_us.geometry_basin.apply(lambda p: p.x) + basin_connections_us.geometry_splitnode.apply(lambda p: p.x))/2
# basin_connections_us.head(3)

In [ ]:
# merge upstream and downstream connections
basin_connections_gdf = basin_connections_us.merge(basin_connections_ds, left_on='mesh1d_nNodes',right_on='mesh1d_nNodes',suffixes=('_out','_in'))

# add coordinate in middle of two nodes upstream and downstream of splitpoint
basin_connections_gdf['x'] = (basin_connections_gdf.geometry_edge_start_node.apply(lambda p: p.x) + basin_connections_gdf.geometry_edge_end_node.apply(lambda p: p.x))/2
basin_connections_gdf['y'] = (basin_connections_gdf.geometry_edge_start_node.apply(lambda p: p.y) + basin_connections_gdf.geometry_edge_end_node.apply(lambda p: p.y))/2
basin_connections_gdf['splitnode_moved'] = gpd.points_from_xy(basin_connections_gdf['x'], basin_connections_gdf['y'])

# add ID for moved split nodes
basin_connections_gdf.insert(0, 'splitnode_moved_id', range(len(basin_connections_gdf)))

splitnodes_moved = basin_connections_gdf[['mesh1d_nNodes','splitnode_moved', 'splitnode_moved_id' ]]
splitnodes_moved = splitnodes_moved.rename(columns={"splitnode_moved":"geometry"})
splitnodes_moved = gpd.GeoDataFrame(splitnodes_moved, geometry='geometry',crs=28992)


# basin_connections_gdf['geometry'] = basin_connections_gdf.apply(lambda row: LineString([row['geometry_basin_in'],row['geometry_basin_out']]), axis=1)
basin_connections_gdf['geometry'] = basin_connections_gdf.apply(lambda row: LineString([row['geometry_basin_out'],row['splitnode_moved'],row['geometry_basin_in']]), axis=1)
# alternative: draw line via split node
# basin_connections_gdf['geometry'] = basin_connections_gdf.apply(lambda row: LineString([row['geometry_basin_in'],row['geometry_splitnode_out'],row['geometry_basin_out']]), axis=1)
basin_connections_gdf = gpd.GeoDataFrame(basin_connections_gdf, geometry='geometry',crs=28992)

basin_connections_gdf = basin_connections_gdf.drop(columns=['geometry_edge_start_node','geometry_edge_end_node', 'splitnode_moved', 'geometry_splitnode_out','geometry_basin_out','start_node_no_out','mesh1d_nEdges_out','end_node_no_out','direction_out','geometry_splitnode_in','geometry_basin_in','start_node_no_in','end_node_no_in','mesh1d_nEdges_in','direction_in'])

In [ ]:
splitnodes_moved.head()

In [ ]:
basin_connections_gdf.head()

In [ ]:
# network.ribasim_edges_gdf = ribasim_edges_gdf
network.basin_connections_gdf = basin_connections_gdf
network.splitnodes_moved_gdf = splitnodes_moved

In [ ]:
network.export_to_geopackage(output_dir=results_dir)

TO DO: create boundary nodes

In [ ]:
import dfm_tools as dfmt
import hydrolib.core.dflowfm as hcdfm

In [ ]:
simulation_name = 'tki_zuthpen_berkel_basis.dsproj_data'

In [ ]:
#Load .bc-file using HydroLib object ForcingModel.

file_bc = f'{base_dir}\d-hydro\\{simulation_name}\FlowFM\input\FlowFM_boundaryconditions1d.bc'
# file_lat = r'C:\Users\NLTAND\OneDrive - Sweco AB\Algemeen-Tessa\Projecten\TKI oppervlaktewatermodule NHI\ribasim_lumping_data\d-hydro\tki_zuthpen_berkel_basis.dsproj_data\FlowFM\input\FlowFM_lateral_sources.bc'
forcingmodel_object = hcdfm.ForcingModel(file_bc)


In [ ]:
bc = pd.DataFrame([forcing.dict() for forcing in forcingmodel_object.forcing])

# convert dictionary with boundary type to columns
bc = pd.concat([bc.drop(['quantityunitpair'], axis=1), pd.DataFrame.from_records(bc['quantityunitpair'])[0].apply(pd.Series)], axis=1)

# merge boundary with nodes
boundaries_gdf = network.nodes_gdf.merge(bc,left_on = 'mesh1d_node_id', right_on = 'name')
boundaries_gdf = boundaries_gdf.drop(columns=['mesh1d_node_x','mesh1d_node_y','offset','factor','vertpositionindex','name', 'comments','datablock'])
boundaries_gdf.insert(0, 'boundary_id', range(len(boundaries_gdf)))

In [ ]:
boundaries_gdf

In [ ]:
network.boundaries_gdf = boundaries_gdf
network.export_to_geopackage(output_dir=results_dir)

TO DO: create connections between basins and boundaries

In [ ]:
network.basins_gdf.head()

In [ ]:
network.boundaries_gdf

In [ ]:

# merge boundaries with edges 
boundary_basin_connections = network.boundaries_gdf.rename(columns={"geometry":"geometry_boundary"})
boundary_basin_connections_us = boundary_basin_connections.merge(network.edges_gdf[['start_node_no', 'end_node_no','mesh1d_nEdges']], left_on='mesh1d_nNodes', right_on='start_node_no')
boundary_basin_connections_ds = boundary_basin_connections.merge(network.edges_gdf[['start_node_no','end_node_no','mesh1d_nEdges']], left_on='mesh1d_nNodes', right_on='end_node_no')

# merge with basins for geometry
# downstream
boundary_basin_connections_ds = boundary_basin_connections_ds.merge(network.basins_gdf[['basin', 'geometry']], left_on='basin', right_on='basin').rename(columns={"geometry":"geometry_basin"})
boundary_basin_connections_ds['boundary_location'] = 'downstream'

boundary_basin_connections_ds['geometry'] = boundary_basin_connections_ds.apply(lambda row: LineString([row['geometry_basin'], row['geometry_boundary']]), axis=1)
boundary_basin_connections_ds = gpd.GeoDataFrame(boundary_basin_connections_ds, geometry='geometry', crs=28992)

# same for upstream
boundary_basin_connections_us = boundary_basin_connections_us.merge(network.basins_gdf[['basin', 'geometry']], left_on='basin', right_on='basin').rename(columns={"geometry":"geometry_basin"})
boundary_basin_connections_us['boundary_location'] = 'upstream'

boundary_basin_connections_us['geometry'] = boundary_basin_connections_us.apply(lambda row: LineString([row['geometry_boundary'], row['geometry_basin']]), axis=1)
boundary_basin_connections_us = gpd.GeoDataFrame(boundary_basin_connections_us, geometry='geometry', crs=28992)

# concat us and ds
boundary_basin_connections_gdf = pd.concat([boundary_basin_connections_us, boundary_basin_connections_ds])
# basin_connections = ribasim_edges_gdf.copy()

boundary_basin_connections_gdf = boundary_basin_connections_gdf.drop(columns=['geometry_boundary','geometry_basin'])
boundary_basin_connections_gdf.insert(0, 'boundary_basin_connection_id', range(len(boundary_basin_connections_gdf)))

boundary_basin_connections_gdf.head()

In [ ]:
network.boundary_basin_connections_gdf = boundary_basin_connections_gdf

Export everything to geopackage

In [ ]:
network.export_to_geopackage(output_dir=results_dir)

Export to ribasim

In [ ]:
network.basins_gdf.head(3)

In [ ]:
network.boundaries_gdf.head(3)

In [ ]:
network.splitnodes_moved_gdf.head(3)

In [ ]:
# network.basin_connections_gdf.head(3)

nodes

In [ ]:
# set id's to node. Start with basins, then boundaries and then moved splitnodes. start with id 1
basins_gdf =network.basins_gdf.copy()
basins_gdf['node_id'] = basins_gdf['basin'] + 1

boundaries_gdf = network.boundaries_gdf.copy()
boundaries_gdf['node_id'] = boundaries_gdf['boundary_id'] + len(network.basins_gdf) +1

splitnodes_moved_gdf = network.splitnodes_moved_gdf.copy()
splitnodes_moved_gdf['node_id'] = splitnodes_moved_gdf['splitnode_moved_id'] + len(network.basins_gdf) + len(network.boundaries_gdf) +1

In [ ]:
# concat basins, boundaries and splitnodes moved 

# oude versie ribasim
# ribasim_node_gdf = pd.concat([basins_gdf.assign(type="Basin"), boundaries_gdf.assign(type="LevelControl"),splitnodes_moved_gdf.assign(type="TabulatedRatingCurve")]) 
ribasim_node_gdf = pd.concat([basins_gdf.assign(type="Basin"), boundaries_gdf.assign(type="LevelBoundary"),splitnodes_moved_gdf.assign(type="TabulatedRatingCurve")]) 


# set node_id as index
ribasim_node_gdf = ribasim_node_gdf.set_index('node_id')

# keep columns geometry and type
ribasim_node_gdf = ribasim_node_gdf[['geometry', 'type']]
ribasim_node_gdf

In [ ]:
# Set up the nodes:

# Make sure the feature id starts at 1: explicitly give an index.
node = ribasim.Node(
    static=ribasim_node_gdf
)
node

edges

In [ ]:
network.basin_connections_gdf.head()

In [ ]:
basin_connections_gdf_us = network.basin_connections_gdf[['splitnode_moved_id', 'basin_out','geometry']]
basin_connections_gdf_us['geometry'] = basin_connections_gdf_us.geometry.apply(lambda x: LineString([x.coords[0], x.coords[1]]))
basin_connections_gdf_us['from_node_id'] = basin_connections_gdf_us['basin_out'] +1
basin_connections_gdf_us['to_node_id'] = basin_connections_gdf_us['splitnode_moved_id'] + len(network.basins_gdf) + len(network.boundaries_gdf) +1
basin_connections_gdf_us.head(3)

In [ ]:
basin_connections_gdf_ds = network.basin_connections_gdf[['splitnode_moved_id', 'basin_in','geometry']]
basin_connections_gdf_ds['geometry'] = basin_connections_gdf_ds.geometry.apply(lambda x: LineString([x.coords[1], x.coords[2]]))
basin_connections_gdf_ds['from_node_id'] = basin_connections_gdf_ds['splitnode_moved_id'] + len(network.basins_gdf) + len(network.boundaries_gdf) +1
basin_connections_gdf_ds['to_node_id'] = basin_connections_gdf_ds['basin_in'] +1
basin_connections_gdf_ds.head(3)




In [ ]:
boundary_basin_connections = network.boundary_basin_connections_gdf[['boundary_id', 'basin','geometry','boundary_location']]


# basin_connections_gdf_ds['geometry'] = basin_connections_gdf_ds.geometry.apply(lambda x: LineString([x.coords[1], x.coords[2]]))
boundary_basin_connections_us = boundary_basin_connections.loc[boundary_basin_connections['boundary_location'] == 'upstream']
boundary_basin_connections_us['from_node_id'] = boundary_basin_connections_us['boundary_id']  + len(network.basins_gdf) +1
boundary_basin_connections_us['to_node_id'] = boundary_basin_connections_us['basin'] +1

boundary_basin_connections_ds = boundary_basin_connections.loc[boundary_basin_connections['boundary_location'] == 'downstream']
boundary_basin_connections_ds['from_node_id'] = boundary_basin_connections_ds['basin'] +1
boundary_basin_connections_ds['to_node_id'] = boundary_basin_connections_ds['boundary_id'] + len(network.basins_gdf) +1

boundary_basin_connections_us.head(3)

In [ ]:
# edges = gpd.GeoDataFrame(
#     index=np.arange(0,100),
#     columns=['basin_in', 'basin_out', 'split_node', 'boundary', 'geometry']
# )
# edges['boundary'] = edges['boundary'] + 47
# edges['split_node'] = edges['split_node'] + 52
# edges#.reset_index()

In [ ]:
# network.basin_connections_gdf.geometry.apply(lambda x: LineString([x.coords[0], x.coords[1]]))

In [ ]:
# Setup the edges:
ribasim_edges = pd.concat([basin_connections_gdf_ds, basin_connections_gdf_us,boundary_basin_connections_us, boundary_basin_connections_ds]) 
ribasim_edges = ribasim_edges[['from_node_id','to_node_id','geometry']].reset_index()
ribasim_edges['from_node_id'].astype(int)

edge = ribasim.Edge(
    static=ribasim_edges
)

ribasim_edges.head(3)

basin

In [ ]:
# # Setup the basins:

# profile = pd.DataFrame(
#     data={
#         "node_id": [0, 0],
#         "storage": [0.0, 1000.0],
#         "area": [0.0, 1000.0],
#         "level": [0.0, 1.0],
#     }
# )
# repeat = np.tile([0, 1], 4)
# profile = profile.iloc[repeat]
# profile["node_id"] = [1, 1, 3, 3, 6, 6, 9, 9]

# # Convert steady forcing to m/s
# # 2 mm/d precipitation, 1 mm/d evaporation
# seconds_in_day = 24 * 3600
# precipitation = 0.002 / seconds_in_day
# evaporation = 0.001 / seconds_in_day


# static = pd.DataFrame(
#     data={
#         "node_id": [0],
#         "drainage": [0.0],
#         "potential_evaporation": [evaporation],
#         "infiltration": [0.0],
#         "precipitation": [precipitation],
#         "urban_runoff": [0.0],
#     }
# )
# static = static.iloc[[0, 0, 0, 0]]
# static["node_id"] = [1, 3, 6, 9]

# basin = ribasim.Basin(profile=profile, static=static)

In [ ]:
profile_data = pd.DataFrame(
    data={
        "node_id": ribasim_node_gdf.loc[ribasim_node_gdf['type']=='Basin'].index.values.tolist()
    }
)

profile_data['storage'] = 3.5
profile_data['area'] = 4.5
profile_data['level'] = 5.5

profile_data.head()


In [ ]:
static_data = pd.DataFrame(
    data={
        "node_id": ribasim_node_gdf.loc[ribasim_node_gdf['type']=='Basin'].index.values.tolist()
    }
)

static_data['drainage'] = 6.5
static_data['potential_evaporation'] = 6.5
static_data['infiltration'] = 6.5
static_data['precipitation'] = 6.5
static_data['urban_runoff'] = 6.5


static_data.head()

In [ ]:
basin = ribasim.Basin(profile=profile_data, static=static_data)

rating curve

In [ ]:
# Discharge: lose 1% of storage volume per day at storage = 1000.0.
static_data = pd.DataFrame(
    data={
        "node_id": ribasim_node_gdf.loc[ribasim_node_gdf['type']=='TabulatedRatingCurve'].index.values.tolist()
    }
)

static_data['level'] = 6.5
static_data['discharge'] = 6.5




rating_curve = ribasim.TabulatedRatingCurve(
    static= static_data,)


static_data.head()

boundary

In [ ]:
static_boundary = boundaries_gdf[['node_id']].copy()
# static_boundary = static_boundary.rename(columns={"boundary_id": "node_id"})
static_boundary
static_boundary['level'] = 6.5

static_boundary

In [ ]:
# boundaries_gdf

In [ ]:
level_boundary = ribasim.LevelBoundary(
    static=static_boundary
)

Export everything to geopackage

In [ ]:
network.export_to_geopackage(output_dir=results_dir)

In [ ]:
# Setup a model:

model = ribasim.Model(
    modelname="ribasim_model",
    node=node,
    edge=edge,
    basin=basin,
    level_boundary=level_boundary,
    # level_control=level_control,
    # linear_level_connection=linear_connection,
    tabulated_rating_curve=rating_curve,
    # fractional_flow=fractional_flow,
    starttime="2020-01-01 00:00:00",
    endtime="2021-01-01 00:00:00",
)

# %%
# Write the model to a TOML and GeoPackage:

model.write(f"{results_dir}/{network.name}")

In [ ]:
# ribasim_node_gdf.loc[ribasim_node_gdf['type']=='Basin']

In [ ]:
ribasim_node_gdf

node

In [ ]:
ribasim_node_gdf

In [ ]:
basin